In [ ]:
"road": [128, 64, 128],
"sidewalk": [244, 35, 232],
"construction": [70, 70, 70],
"tram-track": [192, 0, 128],
"fence": [190, 153, 153],
"pole": [153, 153, 153],
"traffic-light": [250, 170, 30],
"traffic-sign": [220, 220, 0],
"vegetation": [107, 142, 35],
"terrain": [152, 251, 152],
"sky": [70, 130, 180],
"human": [220, 20, 60],
"rail-track": [230, 150, 140],
"car": [0, 0, 142],
"truck": [0, 0, 70],
"trackbed": [90, 40, 40],
"on-rails": [0, 80, 100],
"rail-raised": [0, 254, 254],
"rail-embedded": [0, 68, 63]]

'road': [128, 64, 128],
 'sidewalk': [244, 35, 232],
 'construction_fence': [70, 70, 70]
 'rail_raised_rail_embedded': [0, 254, 254]
 'pole_traffic_light_traffic_sign': [153, 153, 153]
 'sky': [70, 130, 180]
 'human': [220, 20, 60]
 'tram_track_rail_track': [230, 150, 140]
 'car_truck': [0, 0, 142]
 'on_rails': [0, 80, 100]
 'vegetation': [107, 142, 35]
 'trackbed': [90, 40, 40]
 'background_terrain': [152, 251, 152]

                                     [[128, 64, 128], [244, 35, 232], [70, 70, 70], [0, 254, 254], [153, 153, 153], [70, 130, 180],
                                      [220, 20, 60], [230, 150, 140], [0, 0, 142], [0, 80, 100], [107, 142, 35], [90, 40, 40], [152, 251, 152]]

In [ ]:
!pip install transformers

In [ ]:
def print_trainable_parameters(model, print_msg):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for name, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"\n-----{print_msg}......")
    print(
        f"trainable params: {trainable_params / 1e+6:.2f}M || all params: {all_param / 1e+6:.2f}M || trainable%: {100 * trainable_params / all_param:.2f}%"
    )

In [ ]:
import torch
from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation


# load Mask2Former fine-tuned on Cityscapes semantic segmentation
processor = AutoImageProcessor.from_pretrained("facebook/mask2former-swin-tiny-cityscapes-semantic") # tiny-384x384, small - 384x384, large-384x384
model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-tiny-cityscapes-semantic", # tiny-47.41M, small-68.27M, large-215.46M
                                                            ignore_mismatched_sizes=True)

print_trainable_parameters(model, print_msg="Base Model")

for name, params in model.named_parameters():
    if name.startswith("model.pixel_level_module"):
        params.requires_grad = False
print_trainable_parameters(
    model, print_msg="Frozen Model Trainable Params"
)

preprocessor_config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/77.3k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/190M [00:00<?, ?B/s]


-----Base Model......
trainable params: 47.41M || all params: 47.41M || trainable%: 100.00%

-----Frozen Model Trainable Params......
trainable params: 14.46M || all params: 47.41M || trainable%: 30.51%


In [ ]:
import torch

def extract_patches(self, image_pair, samples_per_image, patch_size, stride):
    image_patches = []
    height, width = image_pair['image'].shape[-2], image_pair['image'].shape[-1]
    for i in range(samples_per_image):
        for y in range(0, height - patch_size[0] + 1, stride[0]):
            for x in range(0, width - patch_size[1] + 1, stride[1]):
                cropped_image = crop(image_pair['image'], y, x, patch_size[0], patch_size[1])
                cropped_labels = crop(image_pair['label'], y, x, patch_size[0], patch_size[1])
                image_patches.append({'image': cropped_image, 'label': cropped_labels})
    return image_patches

def crop(image, top, left, height, width):
    return image[..., top:top+height, left:left+width]

Example usage:
image_pair = {'image': torch.randn(1, 3, 1080, 1920), 'label': torch.randn(1, 1, 1080, 1920)}
samples_per_image = 10
patch_size = (256, 256)
stride = (128, 128)
patches = extract_patches(image_pair, samples_per_image, patch_size, stride)


In [ ]:
from torchvision.transforms.functional import crop
import torch

def extract_patches(image_pair, patch_size, stride):
    image_patches = []
    image_height, image_width = image_pair['image'].shape[-2], image_pair['image'].shape[-1]

    # Calculate the number of patches along each dimension
    num_patches_height = (image_height - patch_size[0]) // stride + 1
    print(num_patches_height)
    num_patches_width = (image_width - patch_size[1]) // stride + 1
    print(num_patches_width)
    for i in range(num_patches_height):
        for j in range(num_patches_width):
            top = i * stride
            left = j * stride
            cropped_image = crop(image_pair['image'], top, left, patch_size[0], patch_size[1])
            cropped_labels = crop(image_pair['label'], top, left, patch_size[0], patch_size[1])
            image_patches.append({'image': cropped_image, 'label': cropped_labels})
    return image_patches

image_pair = {'image': torch.randn(1, 3, 1080, 1920), 'label': torch.randn(1, 1, 1080, 1920)}
patch_size = (256, 256)
stride = 128
patches = extract_patches(image_pair, patch_size, stride)


7
14


In [ ]:
len(patches), type(patches), type(patches[0]), patches[0].keys() #98

(98, list, dict, dict_keys(['image', 'label']))

In [ ]:
for i, patch in enumerate(patches):
    print(patch["image"].shape, patch["label"].shape)

torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
torch.Size([1, 3, 256, 256]) torch.Size([1, 1, 256, 256])
torch.Size([1,

In [ ]:
def extract_patches(image_pair, samples_per_image, patch_size):
    image_patches = []
    image_height, image_width = image_pair['image'].shape[-2], image_pair['image'].shape[-1]

    # Calculate the step size to cover the image with 5 non-overlapping patches
    step_height = image_height // samples_per_image
    step_width = image_width // samples_per_image
    print(step_height)
    print(step_width)
    for i in range(samples_per_image):
        for j in range(samples_per_image):
            top = i * step_height
            left = j * step_width
            cropped_image = crop(image_pair['image'], top, left, patch_size[1], patch_size[0])
            cropped_labels = crop(image_pair['label'], top, left, patch_size[1], patch_size[0])
            image_patches.append({'image': cropped_image, 'label': cropped_labels})

    return image_patches

image_pair = {'image': torch.randn(1, 3, 1080, 1920), 'label': torch.randn(1, 1, 1080, 1920)}
patch_size = (512, 512)
patches = extract_patches(image_pair, samples_per_image=5, patch_size=patch_size)

216
384


In [ ]:
len(patches)

25

In [ ]:
for i, patch in enumerate(patches):
    print(patch["image"].shape, patch["label"].shape)

torch.Size([1, 3, 512, 512]) torch.Size([1, 1, 512, 512])
torch.Size([1, 3, 512, 512]) torch.Size([1, 1, 512, 512])
torch.Size([1, 3, 512, 512]) torch.Size([1, 1, 512, 512])
torch.Size([1, 3, 512, 512]) torch.Size([1, 1, 512, 512])
torch.Size([1, 3, 512, 512]) torch.Size([1, 1, 512, 512])
torch.Size([1, 3, 512, 512]) torch.Size([1, 1, 512, 512])
torch.Size([1, 3, 512, 512]) torch.Size([1, 1, 512, 512])
torch.Size([1, 3, 512, 512]) torch.Size([1, 1, 512, 512])
torch.Size([1, 3, 512, 512]) torch.Size([1, 1, 512, 512])
torch.Size([1, 3, 512, 512]) torch.Size([1, 1, 512, 512])
torch.Size([1, 3, 512, 512]) torch.Size([1, 1, 512, 512])
torch.Size([1, 3, 512, 512]) torch.Size([1, 1, 512, 512])
torch.Size([1, 3, 512, 512]) torch.Size([1, 1, 512, 512])
torch.Size([1, 3, 512, 512]) torch.Size([1, 1, 512, 512])
torch.Size([1, 3, 512, 512]) torch.Size([1, 1, 512, 512])
torch.Size([1, 3, 512, 512]) torch.Size([1, 1, 512, 512])
torch.Size([1, 3, 512, 512]) torch.Size([1, 1, 512, 512])
torch.Size([1,

In [ ]:
import numpy as np

def extract_overlapping_patches(image, patch_size, stride):
    img_height, img_width = image.shape[:2]
    num_patches = 6  # You can adjust this as needed

    patches = []
    for i in range(num_patches):
        top = i * stride
        for j in range(num_patches):
            left = j * stride
            patch = image[top : top + patch_size[0], left : left + patch_size[1]]
            patches.append(patch)

    return patches

# Example usage:
image = np.random.rand(1080, 1920)  # Replace with your actual image
patch_size = (512, 512)
stride = 512 #256
all_patches = extract_overlapping_patches(image, patch_size, stride)


In [ ]:
len(all_patches)

36

In [ ]:
!pip install kornia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 12.7 MB/s eta 0:00:00


In [ ]:
import torch
# input = torch.arange(9.).view(1, 1, 3, 3)
from kornia.contrib import extract_tensor_patches
input = torch.randn(1, 3, 1080, 1920)
patches = extract_tensor_patches(input, (512, 512))

In [ ]:
patches.shape

In [ ]:
import torch
import torch.nn.functional as F

class PatchQueue(Dataset):
    '''
    Extract random patches from an image -> put them in queue
    -> feed them to the dataloader
    ref : https://github.com/fepegar/torchio/blob/main/src/torchio/data/queue.py
    '''
    def __init__(self, dataset, max_length,  # Queue Length
                 samples_per_image,  # Patches per image
                 queue_workers=4,  # Num Workers
                 patch_size=[512, 512],  # Patch Size
                 shuffle=False):
        self.dataset = dataset
        self.max_length = max_length
        self.queue_workers = queue_workers
        self.samples_per_image = samples_per_image
        self.shuffle_queue = shuffle
        self.patch_size = patch_size
        self._images_iterable = None
        self.patch_list = list()
        self._num_sampled_images = 0
        self.steps_per_epoch = len(self.dataset) * self.samples_per_image
        self.resize_transform = tvt.Resize(patch_size[::-1])

    def __len__(self):
        return self.steps_per_epoch

    def __getitem__(self, item):
        if not self.patch_list:
            print("[WARN]: Patch List is empty...")
            self._fill()
            self.patch_list.reverse()
        sample_patch = self.patch_list.pop()
        return sample_patch

    @staticmethod
    def _get_first_item(batch):
        return batch[0]

    def initialize_images_iterable(self):
        self._images_iterable = self._get_images_iterable()

    @property
    def images_iterable(self):
        if self._images_iterable is None:
            self.initialize_images_iterable()
        return self.images_iterable

    def _get_images_iterable(self):
        if torch.distributed.is_available() and torch.distributed.is_initialized():
            sampler = DistributedSampler(
                self.dataset,
                num_replicas=torch.distributed.get_world_size(),
                rank=torch.distributed.get_rank(), shuffle=self.shuffle_queue
            )
        else:
            sampler = None
        loader = DataLoader(
            self.dataset,  # num_workers=self.queue_workers, #in multigpu, >0 not wrkng
            batch_size=1, collate_fn=self._get_first_item, pin_memory=True,
            shuffle=self.shuffle_queue, sampler=sampler
        )
        self._num_sampled_images = 0
        return iter(loader)

    def _get_next_image(self):
        try:
            image = next(self._images_iterable)
        except Exception as e:
            print(f"[WARN]:Exception while getting image for patching: {e}")
            self.initialize_images_iterable()
            image = next(self._images_iterable)
        return image

    def extract_non_overlapping_patches_with_padding(self, image_pair, patch_size):
        image = image_pair['image']
        label = image_pair['mask']
        # Check if image size is smaller than patch size
        if orig_height < patch_size[0] or orig_width < patch_size[1]:
            # Resize the image to match the patch size
            image = self.resize_transform(image)
            label = self.resize_transform(label)

        orig_height, orig_width = image.shape[-2], image.shape[-1]
        # Calculate padding needed to make dimensions divisible by patch_size
        pad_height = patch_size[0] - (orig_height % patch_size[0])
        pad_width = patch_size[1] - (orig_width % patch_size[1])
        # Pad the image and label
        padded_image = F.pad(image, (0, pad_width, 0, pad_height), mode="replicate")
        padded_label = F.pad(label, (0, pad_width, 0, pad_height), mode="replicate")
        num_vertical_patches = padded_image.shape[-2] // patch_size[0]
        num_horizontal_patches = padded_image.shape[-1] // patch_size[1]

        image_patches = []
        for i in range(num_vertical_patches):
            for j in range(num_horizontal_patches):
                top = i * patch_size[0]
                left = j * patch_size[1]
                cropped_image = padded_image[:, top: top + patch_size[0], left: left + patch_size[1]]
                cropped_labels = padded_label[:, top: top + patch_size[0], left: left + patch_size[1]]
                cropped_img_name = image_pair['name']
                image_patches.append({'image': cropped_image, 'label': cropped_labels, 'name': cropped_img_name})
        return image_patches

    def _fill(self):
        while True:
            image_pair = self._get_next_image()
            samples_per_image = self.samples_per_image
            patch_size = self.patch_size
            patches = self.extract_patches(
                image_pair, samples_per_image, patch_size
            )
            self.patch_list.extend(patches)
            self._num_sampled_images += 1
            islistfull = len(self.patch_list) >= self.max_length
            if islistfull:
                break

modified_id2label = {
        0: "road", 1: "sidewalk", 2: "construction_fence",
        3: "rail_raised_rail_embedded", 4: "pole_traffic_light_traffic_sign",
        5: "sky", 6: "human", 7: "tram_track_rail_track", 8: "car_truck",
        9: "on_rails", 10: "vegetation", 11: "trackbed",
        12: "background_terrain"
    }

In [ ]:
import torch
import torch.nn.functional as F

def extract_non_overlapping_patches_with_padding(image_pair, patch_size, padding, important_classes=[0, 1, 3, 6, 7, 8, 9, 11]):
        image = image_pair['image']
        label = image_pair['mask']
        # Check if image size is smaller than patch size
        # if orig_height < patch_size[0] or orig_width < patch_size[1]:
        #     # Resize the image to match the patch size
        #     image = self.resize_transform(image)
        #     label = self.resize_transform(label)

        orig_height, orig_width = image.shape[-2], image.shape[-1]
        # Calculate padding needed to make dimensions divisible by patch_size
        if padding:
            pad_height = patch_size[0] - (orig_height % patch_size[0])
            pad_width = patch_size[1] - (orig_width % patch_size[1])
            print(pad_height, pad_width)
            # Pad the image and label
            image = F.pad(image, (0, pad_width, 0, pad_height), mode="constant")
            label = F.pad(label, (0, pad_width, 0, pad_height), mode="constant")
            print(image.shape, label.shape) #1080+456, 1920+128 torch.Size([1, 3, 1536, 2048]) torch.Size([1, 1, 1536, 2048])

        num_vertical_patches = image.shape[-2] // patch_size[0]
        num_horizontal_patches = image.shape[-1] // patch_size[1]

        image_patches = []
        for i in range(num_vertical_patches):
            for j in range(num_horizontal_patches):
                top = i * patch_size[0]
                left = j * patch_size[1]
                cropped_image = image[:, :, top: top + patch_size[0], left: left + patch_size[1]]
                cropped_labels = label[:, :, top: top + patch_size[0], left: left + patch_size[1]]
                cropped_img_name = image_pair['name']
                # Check if any of the unique classes are present in the list of important classes
                # if not any(class_label.item() in important_classes for class_label in torch.unique(cropped_labels)):
                #     continue
                image_patches.append({'image': cropped_image, 'mask': cropped_labels, 'name': cropped_img_name, 'top': top, 'left': left})
        return image_patches

image_pair = {'image': torch.randn(1, 3, 1080, 1920), 'mask': torch.randint(0, 13, size=(1, 1, 1080, 1920)), 'name': '001'}
patch_size = [1024, 1024] #[512, 512]
patches = extract_non_overlapping_patches_with_padding(image_pair, patch_size=patch_size, padding=False)
len(patches), patches[0]["mask"].shape, patches[0]["image"].shape
#padding True
#456 128
# torch.Size([1, 3, 1536, 2048]) torch.Size([1, 1, 1536, 2048])
# (12, torch.Size([1, 1, 512, 512]), torch.Size([1, 3, 512, 512]))
#padding False
#(6, torch.Size([1, 1, 512, 512]), torch.Size([1, 3, 512, 512]))

(1, torch.Size([1, 1, 1024, 1024]), torch.Size([1, 3, 1024, 1024]))

In [ ]:
image_pair = {'image': torch.randn(1, 3, 1080, 1920), 'mask': torch.randn(1, 1, 1080, 1920), 'name': '001'}
patch_size = [512, 512]
patches = extract_non_overlapping_patches_with_padding(image_pair, patch_size=patch_size)
len(patches), patches[0]["mask"].shape, patches[0]["image"].shape

456 128
torch.Size([1, 3, 1536, 2048]) torch.Size([1, 1, 1536, 2048])


(12, torch.Size([1, 1, 512, 512]), torch.Size([1, 3, 512, 512]))

In [ ]:
image_pair = {'image': torch.randn(1, 3, 1080, 1920), 'mask': torch.randn(1, 1, 1080, 1920), 'name': '001'}
patch_size = [1024, 1024]
patches = extract_non_overlapping_patches_with_padding(image_pair, patch_size=patch_size)
len(patches)

968 128
torch.Size([1, 3, 2048, 2048]) torch.Size([1, 1, 2048, 2048])


4

In [ ]:
def reconstruct_original_from_patches(patches, orig_height, orig_width, patch_size):
    # Initialize an empty tensor to hold the reconstructed mask
    reconstructed_mask = torch.zeros((1, 1, orig_height, orig_width), dtype=torch.float32)
    # Get the padded height and widths
    pad_height = patch_size[0] - (orig_height % patch_size[0])
    pad_width = patch_size[1] - (orig_width % patch_size[1])
    # Loop over all patches and copy them into the appropriate locations in the reconstructed mask
    for patch in patches:
        cropped_mask = patch['mask']
        top = patch['top']
        left = patch['left']
        # Unpad the patch to remove the padding that was added during patch extraction
        cropped_mask = cropped_mask[:, :, :patch_size[0] - pad_height, :patch_size[1] - pad_width]
        reconstructed_mask[:, :, top: top + patch_size[0] - pad_height, left: left + patch_size[1] - pad_width] = cropped_mask

    return reconstructed_mask

image_pair = {'image': torch.randn(1, 3, 1080, 1920), 'mask': torch.randn(1, 1, 1080, 1920), 'name': '001'}
patch_size = [512, 512]
patches = extract_non_overlapping_patches_with_padding(image_pair, patch_size=patch_size)

# Assuming you have the list of patches, original image dimensions, and patch size
reconstructed_mask = reconstruct_original_from_patches(patches, orig_height=1080, orig_width=1920, patch_size=[512, 512])

reconstructed_mask.shape

456 128
torch.Size([1, 3, 1536, 2048]) torch.Size([1, 1, 1536, 2048])


torch.Size([1, 1, 1080, 1920])

In [ ]:
def reconstruct_mask_from_patches(patches, orig_height, orig_width, patch_size):
    # Initialize an empty tensor to hold the reconstructed mask
    reconstructed_mask = torch.zeros((1, 1, orig_height, orig_width), dtype=torch.float32)
    pad_height = patch_size[0] - (orig_height % patch_size[0])
    pad_width = patch_size[1] - (orig_width % patch_size[1])
    # Loop over all patches and copy them into the appropriate locations in the reconstructed mask
    for patch in patches:
        cropped_mask = patch['mask']
        top = patch['top']
        left = patch['left']

        # Calculate the valid region within the patch (unpadding)
        valid_height = min(patch_size[0] - pad_height, cropped_mask.shape[-2])
        valid_width = min(patch_size[1] - pad_width, cropped_mask.shape[-1])
        # Copy the unpadded region of the patch to the corresponding location in the reconstructed mask
        reconstructed_mask[:, :, top: top + valid_height, left: left + valid_width] = cropped_mask[:, :, :valid_height, :valid_width]

    return reconstructed_mask


image_pair = {'image': torch.randn(1, 3, 1080, 1920), 'mask': torch.randn(1, 1, 1080, 1920), 'name': '001'}
patch_size = [512, 512]
patches = extract_non_overlapping_patches_with_padding(image_pair, patch_size=patch_size)

# Assuming you have the list of patches, original image dimensions, and patch size
reconstructed_mask = reconstruct_mask_from_patches(patches, orig_height=1080, orig_width=1920, patch_size=[512, 512])
reconstructed_mask.shape

456 128
torch.Size([1, 3, 1536, 2048]) torch.Size([1, 1, 1536, 2048])


torch.Size([1, 1, 1080, 1920])

In [ ]:
import torch

def extract_patches_without_padding(image, patch_size, stride):
    """
    Extract overlapping patches from an image without padding.

    Args:
        image (torch.Tensor): Input image tensor (C, H, W).
        patch_size (tuple): Size of the patches (height, width).
        stride (int): Stride for overlapping patches.

    Returns:
        list: List of image patches.
    """
    image_patches = {}
    height, width = image.shape[-2], image.shape[-1]

    num_vertical_patches = (height - patch_size[0]) // stride + 1
    num_horizontal_patches = (width - patch_size[1]) // stride + 1
    print(num_vertical_patches)
    print(num_horizontal_patches)

    image_patches = []
    for i in range(num_vertical_patches):
        for j in range(num_horizontal_patches):
            top = i * stride
            left = j * stride
            cropped_image = image[:, :, top: top + patch_size[0], left: left + patch_size[1]]
            image_patches.append(cropped_image)

    return image_patches

# Example usage:
image = torch.randn(1, 3, 1080, 1920)  # Replace with your actual image tensor
patch_size = (512, 512)
stride = 512  # Adjust as needed based on your requirements
patches = extract_patches_without_padding(image, patch_size, stride)
print(f"Extracted {len(patches)} overlapping patches.")


2
3
Extracted 6 overlapping patches.


In [ ]:
import torch
from kornia.contrib import ExtractTensorPatches, extract_tensor_patches, combine_tensor_patches

image = torch.randn(1, 3, 1080, 1920)
out = extract_tensor_patches(image, window_size=(512, 512), stride=(512, 512), padding=0) #, allow_auto_padding=False
#combine_tensor_patches(out, original_size=(4, 3), window_size=(3, 3), stride=(3, 3), unpadding=padding)


In [ ]:
out.shape

torch.Size([1, 6, 3, 512, 512])

In [ ]:
reconstruct_img = combine_tensor_patches(out, original_size=(1080, 1920), window_size=(512, 512), stride=(512, 512))

In [ ]:
reconstruct_img.shape

torch.Size([1, 3, 1080, 1920])